In [ ]:
import os
import numpy as np
import pickle as pkl
from openai import OpenAI

## Load Dataset

In [ ]:
indicator = 'sp500'

indicator2name = {
    'sp500': 'S&P 500',
    'nikkei': 'Nikkei 225'
}

In [ ]:
with open('indices.pkl', 'rb') as f:
    indices = pkl.load(f)

with open(f'time_series.pkl', 'rb') as f:
    data = pkl.load(f)

In [ ]:
data_size = data.shape[0]
window_size = 20
print(data_size, window_size, len(indices))

In [ ]:
data_size = len(indices)

num_train = int(data_size * 0.6)
num_test = int(data_size * 0.2)
num_vali = data_size - num_train - num_test

idx_train = np.arange(num_train)
idx_valid = np.arange(num_train, num_train + num_vali)
idx_test = np.arange(num_train + num_vali, num_train + num_vali + num_test)

## Prompt GPT 4

In [ ]:
API_KEY = ''

In [ ]:
system_prompt = f"Your job is to act as a professional financial forecaster. You will be given a time-series data from the past 20 market days. Based on this information, your task is to predict whether the {indicator2name[indicator]} price will decrease by more than 1%, increase by more than 1%, or change minimally in the next market day."
print(system_prompt)

In [ ]:
client = OpenAI(api_key=API_KEY)

In [ ]:
for _i in idx_test:
    i = indices[_i]
    
    data_window = data[i:i+window_size]
    
    s_p_500 = '|'.join([f"{x:.2f}" for x in data_window[:,1]])
    vix = '|'.join([f"{x:.2f}" for x in data_window[:,2]])
    nikkei_225 = '|'.join([f"{x:.2f}" for x in data_window[:,3]])
    ftse_100 = '|'.join([f"{x:.2f}" for x in data_window[:,4]])
    gold_futures = '|'.join([f"{x:.2f}" for x in data_window[:,5]])
    crude_oil_futures = '|'.join([f"{x:.2f}" for x in data_window[:,6]])
    eur_usd = '|'.join([f"{x:.2f}" for x in data_window[:,7]])
    usd_jpy = '|'.join([f"{x:.2f}" for x in data_window[:,8]])
    usd_cny = '|'.join([f"{x:.2f}" for x in data_window[:,9]])
    
    user_prompt = f"Your task is to predict whether the {indicator2name[indicator]} price will:\n"
    user_prompt += f"(1) Decrease: decrease by more than 1%\n(2) Increase: increase by more than 1%\n(3) Neutral: change minimally, between -1% to 1%\nin the next market day. "
    user_prompt += f"Review the time-series data provided for the last {window_size} market days. "
    user_prompt += f"Each time-series consists of daily values separated by a \'|\' token for the following indicators:\n\n"
    user_prompt += f"- S&P 500: {s_p_500}\n- VIX (Volatility Index): {vix}\n- Nikkei 225: {nikkei_225}\n- FTSE 100: {ftse_100}\n- Gold Futures: {gold_futures}\n- Crude Oil Futures: {crude_oil_futures}\n- Exchange rate for EUR/USD: {eur_usd}\n- Exchange rate for USD/JYP: {usd_jpy}\n- Exchange rate for USD/CNY: {usd_cny}\n\n"
    user_prompt += f"Based on this information, predict whether the {indicator2name[indicator]} price will decrease by more than 1%, increase by more than 1%, or otherwise, in the next market day. "
    user_prompt += f"Respond with either \'decrease\', \'increase\', or \'neutral\'. Do not provide any other details. "
    
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
        {
          "role": "system",
          "content": system_prompt
        },
        {
          "role": "user",
          "content": user_prompt
        }
        ],
        temperature=0.7,
        max_tokens=2048,
        top_p=1
    )

    text = response.choices[0].message.content

    with open(f'gpt_predict_time/{i}_{indicator}.txt', 'w') as f:
        f.write(text)